# Домашнее задание 1. Разрешение семантической многозначности (WSD).

При выполнении домашнего задания можно пользоваться тетрадками с семинаров. Пожалуйста, соблюдайте минимальную гигиену кода:
* пишите комментарии, где они уместны;
* избегайте вывода больших текстовых переменных (например, не стоит печатать весь корпус, это существенно усложняет читаемость вашей тетрадки);
* запускайте ячейки последовательно, иначе можно получить невоспроизводимые результаты.

### Формат сдачи задания:

Jupyter-notebook на гитхабе.

### Описание задания:

В этом задании вам нужно будет кластеризовать контексты для неоднозначных слов и проанализировать, как результаты соотносятся с их значениями. Напоминаем, что у нас могут быть как многозначные слова, так и омонимы - для сравнения хорошо взять и те, и другие.

Для сбора данных в п. 1 предполагается использование краулинга. Советуем начать с ручного сбора данных, а затем, если останется время, сделать краулинг словаря. Так вы рискуете меньшим количеством баллов ;)


## **Установки**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install pymystem3 pymorphy2 -q
# !pip install corus -q
# !pip install git+https://github.com/lopuhin/python-adagram.git -q
# !pip install simple-elmo -q

In [6]:
!curl "https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib" > /content/drive/MyDrive/all.a010.p10.d300.w5.m100.nonorm.slim.joblib

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1394M  100 1394M    0     0  32.8M      0  0:00:42  0:00:42 --:--:-- 32.1M


In [ ]:
!wget https://github.com/dialogue-evaluation/morphoRuEval-2017/raw/master/GIKRYA_texts_new.zip -q
!unzip GIKRYA_texts_new.zip
!rm GIKRYA_texts_new.zip
!wget https://github.com/dialogue-evaluation/morphoRuEval-2017/raw/master/RNC_texts.rar -q
!unrar x RNC_texts.rar
!rm RNC_texts.rar

In [ ]:
!wget http://vectors.nlpl.eu/repository/20/196.zip -q
!unzip 196.zip -d 196

In [9]:
import requests
import random
from bs4 import BeautifulSoup
from string import punctuation
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from pymorphy2 import MorphAnalyzer


import adagram
import numpy as np
from simple_elmo import ElmoModel
from sklearn.cluster import KMeans


from corus import load_morphoru_gicrya
from corus import load_morphoru_rnc

## **Задание 1 (1 балл за ручной сбор данных, 1 балл за краулинг)**
Выберите 5 неоднозначных лексем и соберите для них словарные значения (толкования). Можно использовать [МАС](http://feb-web.ru/feb/mas/mas-abc/) или любой другой толковый словарь. Обязательно укажите, каким словарем вы пользовались.

Я буду пользоваться [Грамотой.ру](https://gramota.ru/), а внутри нее Большим толковым словарем русских глаголов (БТСРГ) и Большим толковым словарем русских существительных (БТСРС).

In [78]:
dict_crauler = {'zemlya': 'земля', 'sol': 'соль', 'plyt': 'плыть', 'truba': 'труба', 'luk': 'лук'}

In [82]:
def crauler(word, d_defs):
    r_crauler = requests.get(f'https://gramota.ru/meta/{word}', auth=('user', 'pass'))
    text_crauler = r_crauler.text
    soup_crauler = BeautifulSoup(text_crauler)
    try:  # пробуем искать в БТСРС
        defs_unclear = soup_crauler.find_all("div", {"class": "dict-block dict-id_d50"})[0].find_all("div", {"class": "def-block"})
    except:  # иначе ищем в БТСРС
        defs_unclear = soup_crauler.find_all("div", {"class": "dict-block dict-id_d52"})[0].find_all("div", {"class": "def-block"})
    for defs in defs_unclear:
        if dict_crauler[word] not in d_defs:
            d_defs[dict_crauler[word]] = [defs.get_text().strip()]
        else:
            d_defs[dict_crauler[word]].append(defs.get_text().strip())
    return d_defs

In [83]:
dict_defs = {}
for item in dict_crauler:
    dict_defs = crauler(item, dict_defs)

## **Задание 2 (1 балл)**
Выберите один или несколько корпусов [коллекции Corus](https://github.com/natasha/corus#reference), извлеките оттуда все предложения, где содержатся ваши неоднозначные слова.

In [88]:
dict_sents = {}


def get_sentences(records):
    for record in records:
        hit = ''
        for token in record.tokens:  # я беру претокенизированные корпуса
            if token.lemma in dict_crauler.values():
                hit = token.lemma
        if hit:
            try:  # пробую детокинизировать предложения, чтобы красиво их выводить в конце
                full_sentence = TreebankWordDetokenizer().detokenize([token.text for token in record.tokens])
            except:
                continue
            sentence = []
            for token in record.tokens:
                if not set(token.lemma) & set(punctuation + '«»') and token.lemma:
                    sentence.append(token.lemma)
            if hit in dict_sents:
                dict_sents[hit].append({'text': full_sentence, 'tokens': sentence})
            else:
                dict_sents[hit] = [{'text': full_sentence, 'tokens': sentence}]

In [89]:
path_gicrya = 'gikrya_new_test.out'
records_gicrya = load_morphoru_gicrya(path_gicrya)
get_sentences(records_gicrya)

In [90]:
path_rnc = 'RNCgoldInUD_Morpho.conll'
records_rnc = load_morphoru_rnc(path_rnc)
get_sentences(records_rnc)

## **Задание 3 (2 балла)**
Разбейте полученные контексты по значениям двумя любыми способами (вы можете использовать способы с семинара, заменить там эмбеддинги, реализовать какой-то свой способ).
*Напоминалка*: в случае с AdaGram вы сразу получаете какой-то номер значения, с ELMo и другими контекстуальными эмбеддингами — нужно кластеризовать.

### *AdaGram*

In [44]:
vm = adagram.VectorModel.load('/content/drive/MyDrive/all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

In [ ]:
for ambiguous in dict_sents:
    for i, sentence in enumerate(dict_sents[ambiguous]):
        means = vm.disambiguate(ambiguous, sentence['tokens'])
        val = np.argmax(means)
        dict_sents[ambiguous][i]['adagram'] = val

### *ELMo*

In [ ]:
my_model = ElmoModel()
my_model.load("196")

In [96]:
def get_elmo_vectors(word, tokens, model):
    all_vectors = my_model.get_elmo_vectors(tokens)
    word_vecs = []
    errors = []
    for i in range(len(tokens)):
        try:
            word_vecs.append(all_vectors[i][tokens[i].index(word)])
        except ValueError:
            errors.append(i)
    cluster = KMeans(n_clusters=len(dict_defs[word]), random_state=0, n_init='auto')
    cluster.fit(word_vecs)
    labels = cluster.labels_
    for i in errors:
      labels = labels.insert(i, -1)
    return labels

In [97]:
for ambiguous in dict_sents:
    tkns = [d_amb['tokens'] for d_amb in dict_sents[ambiguous]]
    labs = get_elmo_vectors(ambiguous, tkns, my_model)
    for j, d_amb in enumerate(dict_sents[ambiguous]):
        d_amb['elmo'] = labs[j]
        dict_sents[ambiguous][j] = d_amb

## **Задание 4 (1 балл)**
Возьмите словарные толкования для каждого значения и припишите им значение автоматически теми же способами, что и в п.2.

In [98]:
morph = MorphAnalyzer()
token = RegexpTokenizer('\w+')


def normalize(text):
    words = [morph.parse(word)[0].normal_form for word in tokenize(text) if word]
    return words


def tokenize(text):
    return token.tokenize(text)

### *AdaGram*

In [ ]:
def_preds = {}

for word in dict_defs:
    def_preds[word] = []
    for definition in dict_defs[word]:
        definition_norm = normalize(definition)
        means = vm.disambiguate(word, definition_norm)
        val = np.argmax(means)
        def_preds[word].append({'def': definition, 'adagram': val})

### *ELMo*

In [100]:
for word in dict_defs:
    tkns = [normalize(definition) for definition in dict_defs[word]]
    for i, tkn in enumerate(tkns):
        tkn.insert(0, word)
        tkns[i] = tkn
    lbls = get_elmo_vectors(word, tkns, my_model)
    for j, dct in enumerate(def_preds[word]):
        dct['elmo'] = lbls[j]
        def_preds[word][j] = dct

## **Задание 5 (1 балл)**
Выберите по 2-3 значения для слова и по 5 контекстов, которые автоматически были отнесены к этим значениям. Разметьте контексты: совпадает ли значение слова в контексте со словарным значением.

In [131]:
for ambiguous in dict_sents:
    rand_dict_1 = random.choice(def_preds[ambiguous])
    rand_dict_2 = random.choice(def_preds[ambiguous])
    while rand_dict_2 == rand_dict_1:
        rand_dict_2 = random.choice(def_preds[ambiguous])

    five_ada_1 = [dict_ambi for dict_ambi in dict_sents[ambiguous] if dict_ambi['adagram'] == rand_dict_1['adagram']][:5]
    five_elmo_1 = [dict_ambi for dict_ambi in dict_sents[ambiguous] if dict_ambi['elmo'] == rand_dict_1['elmo']][:5]
    five_ada_2 = [dict_ambi for dict_ambi in dict_sents[ambiguous] if dict_ambi['adagram'] == rand_dict_2['adagram']][:5]
    five_elmo_2 = [dict_ambi for dict_ambi in dict_sents[ambiguous] if dict_ambi['elmo'] == rand_dict_2['elmo']][:5]

    print('Слово:', ambiguous)

    print('Определение:', rand_dict_1['def'])\

    print(f'AdaGram ({rand_dict_1["adagram"]})')
    for i, dct in enumerate(five_ada_1):
        print(f"{i + 1}. {dct['text']}")

    print(f'ELMo ({rand_dict_1["elmo"]})')
    for i, dct in enumerate(five_elmo_1):
        print(f"{i + 1}. {dct['text']}")

    print('Определение:', rand_dict_2['def'])

    print(f'AdaGram ({rand_dict_2["adagram"]})')
    for i, dct in enumerate(five_ada_2):
        print(f"{i + 1}. {dct['text']}")

    print(f'ELMo ({rand_dict_2["elmo"]})')
    for i, dct in enumerate(five_elmo_2):
        print(f"{i + 1}. {dct['text']}")

Слово: земля
Определение: Земная поверхность — суша, твердь, имеющая в своей основе почву, камень и др. породы (в отличие от водного или воздушного пространства).
AdaGram (2)
1. Это значит, что если самый жаркий день над землей был, скажем 25 июля, то на глубине трех метров он наступит лишь 9 октября.
2. если он снова владеет мячом, после того как бросил его на землю и к мячу не прикасался любой другой игрок —: считается, что вратарь контролирует мяч, прикасаясь к нему любой частью запястий или рук, за исключением тех случаев, когда мяч случайно отскакивает от вратаря, например, после взятия пробитого мяча.
3. Он не зря считается в авиации опасным явлением погоды и коварен тем, что сверху, с большой высоты наземные ориентиры, в том числе посадочные огни, просматриваются хорошо, а непосредственно перед землей они внезапно теряются.
4. Его внимание полностью было приковано к земле.
5. Бесперебойное питание проводника, укреплённого на стеклянных изоляторах, отстоящих от земли на высоту по

Слово: земля

Определение: Земная поверхность — суша, твердь, имеющая в своей основе почву, камень и др. породы (в отличие от водного или воздушного пространства).

AdaGram (2)
1. Это значит, что если самый жаркий день над землей был, скажем 25 июля, то на глубине трех метров он наступит лишь 9 октября.
2. если он снова владеет мячом, после того как бросил его на землю и к мячу не прикасался любой другой игрок —: считается, что вратарь контролирует мяч, прикасаясь к нему любой частью запястий или рук, за исключением тех случаев, когда мяч случайно отскакивает от вратаря, например, после взятия пробитого мяча.
3. Он не зря считается в авиации опасным явлением погоды и коварен тем, что сверху, с большой высоты наземные ориентиры, в том числе посадочные огни, просматриваются хорошо, а непосредственно перед землей они внезапно теряются.
4. Его внимание полностью было приковано к земле.
5. Бесперебойное питание проводника, укреплённого на стеклянных изоляторах, отстоящих от земли на высоту пояса среднего человека, обеспечивалось бы электрогенераторами с подведёнными к ним гидроприводами.

ELMo (3)
1. Ранней весной 1942 года землю, пригодную под пашню, передали для использования частным лицам.
2. ~~Немалую часть своей земли они завоевали у моря, немалую часть своих рынков они отвоевали у соседей, и всю свою промышленность они создали сами.~~
3. ~~Загородный дом находится в общей долевой собственности, земля оформлена.~~
4. Земля здесь была густо покрыта соломой.
5. ~~Бизнесмены из России в_первую_очередь пытаются в Абхазии купить земли побольше и подешевле, кто-то собирается заняться производством стройматериалов для будущих строек Сочи - 2014.~~

Определение: То же, что государство.

AdaGram (1)
1. Моя земля больше не будет моей.
2. Его тело, тоже, нуждалось в отдыхе, том, который могла дать только породившая их расу земля.
3. Тёмные пришли в эти земли.
4. Потом ты будешь обитать в стране снов, эльфийской земле, где все возможно, стоит лишь пожелать, и глупцы называют ту землю раем.
5. Вся их жизнь была сплошным бегством, они меняли небо и землю, траву и ветер, в то же время не изменялись сами.

ELMo (0)
1. ~~Наконец шасси коснулись земли, и самолет покатился по ровной пустынной полосе, пролегшей между посадочными огнями —, к сияющему светом скоплению зданий.~~
2. ~~Юноша воспользовался секундой падения, для того чтобы собраться и, перекатившись по земле, вскочить на ноги.~~
3. ~~Он сидел на земле, поджав под себя одну ногу, и осторожно растирал ее бедро.~~
4. ~~Я несся, едва касаясь земли, и обогнал всех остальных настолько, что меня встретили приветственные выкрики.~~
5. ~~Она собралась уже улечься прямо на земле, но замешкалась.~~

Слово: лук

Определение: Метательное холодное оружие, представляющее собой деревянную полку, согнутую в дугу, концы которой стянуты нитью из ткани или жил животных, предназначенное для метания стрел и служащее для поражения живой силы противника.

AdaGram (1)
1. Он научился понимать язык жестов и картинное письмо, а когда ему исполнилось восемь лет и он уже научился хорошо ездить на коне и стрелять из лука, отец впервые взял его на охоту.
2. У меня есть опыт стрельбы из автомата и из лука, не более, так что я тут бесполезен)
3. хочу пострелять из лукаааа!!!!
4. после парашюта сразу пойти пострелять из лука ..
5. Можно пострелять из лука во время прыжка!

ELMo (2)
1. ~~Готовится мясной / куриный бульон как обычно, с добавлением репчатого лука на определенной стадии.~~
2. ~~Лук измельчают и перемешивают с нарезанным мелкими кусочками мясом.~~
3. ~~Лук (много) обжаривается в растительном или оливковом масле.~~
4. ~~На днях, доработавшись до чёрных мушек, я пошёл в чайхану--это обыкновенный трактир, по-нашему " павильон " или " забегаловка ", где, кроме водки, воды, пива, креплёного красного вина, крутых яиц, лука, пампушек, рыбных консервов, бывают иногда чай и плов из мяса неопределённого происхождения, боюсь, что верблюжьего,-- и, чтоб ободриться, выпил рюмки две дрянной ашхабадской водки.~~
5. ~~Плоские кусочки продукта обложить мелко нарезанными овощами, луком, посолить по вкусу, добавить немного масла, зелени, специй.~~

Определение: Инвентарь: ручное оружие для метания стрел в виде дуги, стянутой тетивой (бечевой, струной, стягивающей концы лука), употребляемое в стрельбе из лука.

AdaGram (1)
1. Он научился понимать язык жестов и картинное письмо, а когда ему исполнилось восемь лет и он уже научился хорошо ездить на коне и стрелять из лука, отец впервые взял его на охоту.
2. У меня есть опыт стрельбы из автомата и из лука, не более, так что я тут бесполезен)
3. хочу пострелять из лукаааа!!!!
4. после парашюта сразу пойти пострелять из лука ..
5. Можно пострелять из лука во время прыжка!

ELMo (1)
1. Он опустил лук и обернулся, дружески улыбнувшись.
2. Вскоре после случившегося брат вдовца взял лук и отправился в лес.
3. Он научился понимать язык жестов и картинное письмо, а когда ему исполнилось восемь лет и он уже научился хорошо ездить на коне и стрелять из лука, отец впервые взял его на охоту.
4. Самые мудрые говорили при этом, что теперь надо готовиться к худшему и держать луки да копья наготове.
5. Одна из сильных ног девушки вытянулась вдоль спины чудовища, торс луком выгнулся вперед.

Слово: плыть

Определение: Двигаться в определенном направлении легко, как бы не прилагая усилий, подобно тому как передвигается рыба в воде (о человеке)

AdaGram (2)
1. ~~Другими словами, в данном случае лед в воде не плывет.~~
2. Я словно плыл — не по воде, нет.
3. ~~В море плывут, чтоб наесться скорей.~~
4. ~~Согласен, но это вовсе не значит, что надо НАПЛЕВАВ на ДОБРО, СПРАВЕДЛИВОСТЬ, ЧУВСТВЕННОСТЬ и прочие положительные качества человека, выкинуть всех за борт лодки и пусть плывут ...~~
5. ~~« надо НАПЛЕВАВ на ДОБРО, СПРАВЕДЛИВОСТЬ, ЧУВСТВЕННОСТЬ и прочие положительные качества человека выкинуть всех за борт лодки и пусть плывут...»~~

ELMo (0)
1. ~~Другими словами, в данном случае лед в воде не плывет.~~
2. ~~Плыви, ублюдок, на все четыре стороны.~~
3. Я словно плыл — не по воде, нет.
4. ~~В море плывут, чтоб наесться скорей.~~
5. ~~Согласен, но это вовсе не значит, что надо НАПЛЕВАВ на ДОБРО, СПРАВЕДЛИВОСТЬ, ЧУВСТВЕННОСТЬ и прочие положительные качества человека, выкинуть всех за борт лодки и пусть плывут ...~~

Определение: Передвигаться в воде с помощью определенных движений тела, загребая, выталкивая воду назад (о человеке и животных, не живущих в воде), а также передвигаться по поверхности или в глубине воды при помощи специальных приспособлений (о корабле, лодке и т.п.)

AdaGram (4)
1. Разгребая перед собой нечистоты, вынесенные Золотухой, натуральное дерьмо, плавающее вокруг дебаркадера, плыли вдаль будущие граждане Страны Советов.
2. Испытуемый В .: рис . 1--кораблик, плывущий по волнам; рис . 2--кораблик в море (напряжение, покусывает губы), вижу нос.
3. Две птицы (крутит карандаш в руках), рис . 4 --(двигает рисунок ближе к себе) корабль плывёт...и самолёт; рис . 5--корабль плывёт, три птицы и самолёт; рис . 6--корабль, три птицы и самолёт; рис . 7 --(двигает рисунок ещё ближе к себе) какая-то символика: рис . 8 --(растерянно) небо...облака.
4. Неправильно, что есть дым! (поправляет волосы, улыбается); рис . 5--плывущий пароход, спокойное море, хорошая погода...морское животное, летящие птицы (двигает рисунок ближе к себе и подпирает голову рукой)... из земли червяк ползущий; рис . 6--плывущий пароход, море, хорошая погода, прыгающий дельфин...две летящие птицы (разглядывает рисунок с разных сторон, выражение отчаяния на лице)... хочется фигуру пониже; рис . 7--пароход, море ...(берёт карандаш и начинает судорожно показывать им на рисунке то, что видит), либо дерево, либо коряги (водит карандашом по лицу), либо город, либо животное (рассматривает только верхнюю часть рисунка)... похоже на русалку или человека (гадает, что же это может быть); рис . 8--пароход ...

ELMo (3)

1. ~~Казалось, что лампа сама плывет в воздухе.~~
2. Оба мои товарища плыли рядом со мной.
3. ~~Черная свеча плыла над ней, освещая путь.~~
4. Хозяин словно плыл в ней, цепляя подолом снег.
5. Вот уже три дня они плыли под парусами при благоприятном ветре, вставая на якорь только по ночам.

Слово: соль

Определение: В химии: соединение, в котором водород кислоты замещен металлом.

AdaGram (1)
1. Сначала доведем до нормы соль.
2. На развилине её был укреплён большой кусок каменной соли.
3. Там добавление физиологически кислых солей только улучшает почвенные условия.
4. 100 минералов, солей и микроэлементов из таинственных глубин Мёртвого моря, необходимых для того, чтобы Ваша кожа не утратила сияния молодости!
5. ~~Массаж с солью~~

ELMo (2)
1. ~~Ладан — сжигается на угле Порошок — рассыпается по углам комнаты, помещения, добавляется в магический защитный мешочек вместе с другими ингредиентами Соль — для ванны, полоскания белья, мытья пола и для гри-гри.~~
2. ~~Салат очень красиво выглядит и уверяю вас--никто не сможет определить ингредиенты этого блюда) Надо: Свежие огурцы, крабовые палочки, салат, соль - перец, майонез, несколько капель лимонного сока, зелень.~~
3. ~~15 - 20 крупных шампиньонов (или столько, сколько нужно для гостей), 200 - 250 г твёрдого сыра, зелень, майонез, приправы, соль, перец.~~
4. ~~Сыр измельчить, добавить в него мелко нарезанные ножки, зелень, майонез, соль, перец, приправы по вкусу.~~
5. Все орехи богаты минеральными солями, кальцием, фосфором и магнием, недостаток этих полезных веществ в организме может привести к болезням сердца, нервозности, плохому самочувствию, вызвать чувство необоснованного беспокойства.

Определение: Белое кристаллическое вещество с острым характерным вкусом, соединение натрия и хлора, растворяющееся в воде и используемое как приправа в пищу.

AdaGram (1)
1. ~~Сначала доведем до нормы соль.~~
2. На развилине её был укреплён большой кусок каменной соли.
3. ~~Там добавление физиологически кислых солей только улучшает почвенные условия.~~
4. 100 минералов, солей и микроэлементов из таинственных глубин Мёртвого моря, необходимых для того, чтобы Ваша кожа не утратила сияния молодости!
5. Массаж с солью

ELMo (1)
1. Соль убила бы любую воду, если б колодцы не высохли сами.
2. ~~Вот в чем соль земной жизни.~~
3. А нам до этого говорили, что в Турции хлебоносные деревья и сахар из земли прямо, как соль, добывают.
4. " А что же нам говорили, что в Турции сахар из земли роют, как соль, и хлебные деревья растут?"--не унимаются наши.
5. " Да нам говорили,--объясняют наши,--что в Турции сахар прямо из земли роют, как соль, и хлебные деревья растут.

Слово: труба

Определение: Предмет удлиненной цилиндрической формы, обычно из железа, по которой движется жидкость, пар, газ и т.п.

AdaGram (0)
1. Труба взорвалась фонтаном разлетающихся во все стороны кирпичей и бетона.
2. По этим трубам, защищенным свинцовой и керамической оболочками, струился водяной пар, охлаждавший раскаленное чрево реактора.
3. Отсос из труб агрофирмы ведётся через пожарный рукав.
4. Поздравительные букеты, дорогая оптика (бинокли, подзорные трубы), ружья--здесь логистика не имеет большого значения, поскольку цена единицы каждого товара или услуги несопоставима со стоимостью доставки, а ассортимент не столь велик, чтобы обзаводиться складом.
5. В книге приведены и характеристики лайнера; длина--243 метра, водоизмещение--45 тысяч тонн, мощность двигателей--75 тысяч лошадиных сил, максимальная скорость--25 узлов, а также некоторые другие подробности: четыре трубы, три винта, две мачты.

ELMo (2)
1. Развернув послушного милиционера затылком к стене тамбура, он охватил одним замком наручников запястье, а вторым приторочил его к трубе стоп-крана.
2. А еще можно догнать беднягу по пути домой и врезать ему по голове обрезком трубы.
3. Тяжело бухало по крыше, шарило у трубы.
4. ~~« Ну, всё, Новосельцев, Ваше дело труба . »,~~
5. Незнакомец сопровождал его, задевая плечом водосточные трубы.

Определение: Медный духовой инструмент, представляющий собой трубку, согнутую в несколько раз и завершающуюся небольшим раструбом, узкий конец которой снабжен мундштуком, издающий сильные звуки; простейшие образцы были известны до нашей эры, с XVII века трубу начали применять в оркестрах.

AdaGram (2)
1. ~~Я одной чурбачок в трубу спустил, так, веришь ли, два часа за мной гонялась.~~
2. ~~Пока матросы и кое-кто из офицеров отправились приводить в состояние готовности взрыв - трубы, команда на парусах занялась снастями.~~
3. Уличные музыканты наигрывали на флейтах, арфах и трубах, аккомпанируя иногда жонглеру или акробату, положив у себя перед ногами шапку для звонких монет.
4. ~~« Ну, всё, Новосельцев, Ваше дело труба . »,~~
5. Ухнул барабан, загудели трубы и публика повалила к эстрадам.

ELMo (1)
1. Это такая полая труба и в неё дуют, а он для акустического эффекта просто выдувает воздух в замкнутое пространство.
2. ~~Над толстой прозрачной крышей бака трубы эти широко расходились и соединялись третьей горизонтальной трубой со множеством вставленных в нее коротких открытых трубок.~~
3. ~~Наземная команда уже подсоединяет герметичные трубы - проходы к нашим шлюзам, и когда они закончат, вы сможете покинуть корабль.~~
4. ~~Я одной чурбачок в трубу спустил, так, веришь ли, два часа за мной гонялась.~~
5. ~~Но тем же попаданием ломает трубу с водой, которой этот пожар заливает.~~

## **Задание 6 (1 балл)**
Оцените accuracy — в какой доле контекстов автоматически определенно значение для контекста совпадает с автоматически определенным значением толкования.

In [132]:
wrong = 39
total = 99
accuracy = (total - wrong) / total

In [136]:
print('Общее accuracy равно', round(accuracy, 2))

Общее accuracy равно 0.61


In [137]:
ada_total = 49
elmo_total = 50
ada_wrong = 9
elmo_wrong = 30
ada_accuracy = (ada_total - ada_wrong) / ada_total
elmo_accuracy = (elmo_total - elmo_wrong) / elmo_total

In [138]:
print('Accuracy для AdaGram равно', round(ada_accuracy, 2))
print('Accuracy для ELMo равно', round(elmo_accuracy, 2))

Accuracy для AdaGram равно 0.82
Accuracy для ELMo равно 0.4


## **Задание 7 (максимум 2 балла)**
Сравните результаты двух методов WSD, объясните, почему результаты совпадают или расходятся (и покажите на конкретных примерах).

Как можно заметить по accuracy в предыдущей клеточке для двух методов WSD, ELMo сильно проигрывает AdaGram. Давайте разбираться почему!

Как мне кажется, основной причиной является тот факт, что ELMo, в отличие от AdaGram, не предназнакачен для WSD и все портит KMeans, так как он обязательно разбивает на строго $n$ групп в том числе определения. А определения приписываются к группам вообще криво (приходится добавлять определяемое слово в начале...).

Самую частотную проблему ELMo можно определить как "все (или почти все) не туда" (см. первое размеченное определение лука, второе размеченное определение земли или второе размеченное определение трубы). Как мне кажется, оно возникает из-за попытки KMeans развести по разным группам очень близкие определения, что приводит к тому, что она разводит их слишком сильно, ошибаясь в соотнесении классу. Таким образом класс выделен правильно, но приписан не тому определению. У AdaGram таких проблем не возникает, поэтому accuracy заметно выше.

Вторая проблема: близкие определения, в которых есть уточнения, о ком речь (например, *о человеке* в первом определении *плыть*). Такие тонкости плохо считывают оба метода.

Третья проблема: фразеология и устойчивые сочетания. Непонятно, к какому определению вообще относить такие примеры как *Ну, всё, Новосельцев, Ваше дело труба* или *Вот в чем соль земной жизни.*, для которых нет определений. Такие тонкости также не считывают оба метода.

